<a href="https://colab.research.google.com/github/ds4geo/ds4geo/blob/master/WS%202020%20Course%20Notes/Session%209.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Session 9

Supervised ML and Deep learning

# Introduction to Supervised Machine Learning
* Learning classification or regression from data with labels
* aim to learn general rules which can be applied on other data
* many algorithms are black boxes - the "rules" are difficult or impossible to know or understand

* examples

youtube CGP grey

# Introduction to rock geochem database

* Can we use ML to predict the rock types based on the bulk chemistry?

# Excercise - load and analyse rock geochem data

* load data
* some simple viz
* PCA of data

# Machine Learning Algorithms: Random Forest
* explain decision tree
* under vs over fitting
* random forest as solution to over-fitting
* explain how it works
 * forest/ensemble of imperfect trees better at generalising